# Scratchwork

This notebook is only used for trying out ideas.

In [10]:
import algebras as alg
import json
import os
import numpy as np
import itertools as it

In [11]:
# Path to this repo
aa_path = os.path.join(os.getenv('PYPROJ'), 'abstract_algebra')

# Path to a directory containing Algebra definitions in JSON
alg_dir = os.path.join(aa_path, "Algebras")

## Groups for Testing

In [12]:
d4_path = os.path.join(alg_dir, "d4_dihedral_group_on_4_vertices.json")
!cat {d4_path}

{"type": "Group",
 "name": "D_4",
 "description": "Dihedral group on four vertices",
 "element_names": ["e", "r", "r^2", "r^3", "f", "fr", "r^2f", "rf"],
 "alt_elem_names": ["()", "(0 1 2 3)", "(0 2)(1 3)", "(0 3 2 1)",
                    "(0 1)(2 3)", "(1 3)", "(0 3)(1 2)", "(0 2)"],
 "mult_table": [[0, 1, 2, 3, 4, 5, 6, 7],
                [1, 2, 3, 0, 7, 4, 5, 6],
                [2, 3, 0, 1, 6, 7, 4, 5],
                [3, 0, 1, 2, 5, 6, 7, 4],
                [4, 5, 6, 7, 0, 1, 2, 3],
                [5, 6, 7, 4, 3, 0, 1, 2],
                [6, 7, 4, 5, 2, 3, 0, 1],
                [7, 4, 5, 6, 1, 2, 3, 0]]
}

In [13]:
d4 = alg.Group(d4_path)
d4.print_info()


Group : D_4 : Dihedral group on four vertices
  Element Names: ['e', 'r', 'r^2', 'r^3', 'f', 'fr', 'r^2f', 'rf']
  Is Abelian? False
  Inverses:  (** - indicates that it is its own inverse)
    inv(e) = e   **
    inv(r) = r^3 
    inv(r^2) = r^2   **
    inv(r^3) = r 
    inv(f) = f   **
    inv(fr) = fr   **
    inv(r^2f) = r^2f   **
    inv(rf) = rf   **
Element Orders:
{1: ['e'], 2: ['r^2', 'f', 'fr', 'r^2f', 'rf'], 4: ['r', 'r^3']}
  Is associative? True
  Cayley Table:
       e     r   r^2   r^3     f    fr  r^2f    rf
       r   r^2   r^3     e    rf     f    fr  r^2f
     r^2   r^3     e     r  r^2f    rf     f    fr
     r^3     e     r   r^2    fr  r^2f    rf     f
       f    fr  r^2f    rf     e     r   r^2   r^3
      fr  r^2f    rf     f   r^3     e     r   r^2
    r^2f    rf     f    fr   r^2   r^3     e     r
      rf     f    fr  r^2f     r   r^2   r^3     e


## Order of an Element

In [14]:
d4.element_order('r')

4

In [15]:
d4.element_orders()

{'e': 1, 'r': 4, 'r^2': 2, 'r^3': 4, 'f': 2, 'fr': 2, 'r^2f': 2, 'rf': 2}

In [16]:
d4.element_orders(True)

{1: ['e'], 4: ['r', 'r^3'], 2: ['r^2', 'f', 'fr', 'r^2f', 'rf']}

In [17]:
from pprint import pprint

pprint(d4.element_orders(True))

{1: ['e'], 2: ['r^2', 'f', 'fr', 'r^2f', 'rf'], 4: ['r', 'r^3']}


In [18]:
help(pprint)

Help on function pprint in module pprint:

pprint(object, stream=None, indent=1, width=80, depth=None, *, compact=False)
    Pretty-print a Python object to a stream [default is sys.stdout].



## Experiments in Finding Generator Sets

In [19]:
d4.element_names

['e', 'r', 'r^2', 'r^3', 'f', 'fr', 'r^2f', 'rf']

In [20]:
aa = set(d4.element_names[0])
bb = set(d4.element_names[1:])

In [21]:
aa | bb

{'e', 'f', 'fr', 'r', 'r^2', 'r^2f', 'r^3', 'rf'}

In [22]:
def minimum_generators(grp):
    gens = set()
    n = len(grp.element_names)
    init = set(grp.element_names[0])
    remain = set(grp.element_names[1:])
    for i in range(1, n - 2):
        for combo in it.combinations(remain, i):
            candidate = init | set(combo)
            clo = grp.closure(candidate)
            if len(clo) == n:
                gens.add(frozenset(candidate))
    gens_as_lists = list(map(lambda x: list(x), gens))
    min_gen_len = min({len(gen) for gen in gens_as_lists})
    min_gens = [gen for gen in gens_as_lists if len(gen) == min_gen_len]
    return min_gens

In [23]:
def minimum_generators(grp):
    gens = set()
    n = len(grp.element_names)
    remain = set(grp.element_names[1:])
    for i in range(2, n - 1):
        for combo in it.combinations(remain, i):
            candidate = set(combo)
            clo = grp.closure(candidate)
            if len(clo) == n:
                gens.add(frozenset(candidate))
    gens_as_lists = list(map(lambda x: list(x), gens))
    min_gen_len = min({len(gen) for gen in gens_as_lists})
    min_gens = [gen for gen in gens_as_lists if len(gen) == min_gen_len]
    return min_gens

In [24]:
minimum_generators(d4)

[['r', 'r^2f'],
 ['r^2f', 'r^3'],
 ['f', 'rf'],
 ['fr', 'r^3'],
 ['fr', 'r'],
 ['fr', 'r^2f'],
 ['f', 'r^3'],
 ['r', 'rf'],
 ['r', 'f'],
 ['rf', 'r^3'],
 ['r^2f', 'rf'],
 ['fr', 'f']]

In [25]:
grp = d4
gen = ['e', 'r', 'fr']

def swap_pair(pair):
    return (pair[1], pair[0])

all_pairs = []
elems = set(gen)
for pair in it.combinations(gen, 2):
    all_pairs.append(pair)
    pair_prod = d4.mult(*pair)
    
    print(pair, pair_prod)
    swap = swap_pair(pair)
    all_pairs.append(swap)
    swap_prod = d4.mult(*swap)
    print(swap, swap_prod)
print(all_pairs)

('e', 'r') r
('r', 'e') r
('e', 'fr') fr
('fr', 'e') fr
('r', 'fr') f
('fr', 'r') r^2f
[('e', 'r'), ('r', 'e'), ('e', 'fr'), ('fr', 'e'), ('r', 'fr'), ('fr', 'r')]
